## PIIC - predecir ozono atmosferico
---
**Obj:** predecir el ozono atmosférico $O_3$ máximo del dia siguiente, utilizando la información de químicos de los dias anteriores.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras,gc,time
from keras.layers import *
from keras.models import Sequential,Model

Using TensorFlow backend.


### Cargar datos

In [2]:
folder = "../proyecto/datasets/"
df_independencia_verano = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-verano.csv")
df_independencia_invierno = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-invierno.csv")

df_condes_verano = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-verano.csv")
df_condes_invierno = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-invierno.csv")

df_independencia_verano.head()

,registered_on,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
0,1997-11-01 01:00:00,2.7,63.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1997-11-01 02:00:00,2.6,54.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,1997-11-01 03:00:00,2.2,53.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,1997-11-01 04:00:00,2.5,65.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,1997-11-01 05:00:00,2.3,118.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [5]:
## ultimo año como test
df_condes_verano_train = df_condes_verano[(df_condes_verano["registered_on"]<"2017")]
df_condes_invierno_train = df_condes_invierno[(df_condes_invierno["registered_on"]<"2017")]
df_condes_verano_test = df_condes_verano[df_condes_verano["registered_on"]>="2017"]
df_condes_invierno_test = df_condes_invierno[df_condes_invierno["registered_on"]>="2017"]

df_condes_verano_train.tail()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69390,2016-12-31 14:00:00,NaN,0.553417,53.0,24.0,7.0400,3.2450,10.28500,NaN,NaN,41.8333,26.6333,NaN,NaN,NaN,NaN,72.5233
69391,2016-12-31 15:00:00,NaN,0.467584,22.0,21.0,4.4825,2.7500,7.23250,NaN,NaN,39.3333,26.3167,NaN,NaN,NaN,NaN,58.9725
69392,2016-12-31 16:00:00,NaN,0.459000,16.5,13.0,3.4925,2.9975,6.49000,NaN,NaN,37.3333,26.0583,NaN,NaN,NaN,NaN,50.3033
69393,2016-12-31 17:00:00,NaN,0.459000,15.0,14.0,3.9050,3.4100,7.31500,NaN,NaN,36.3333,25.1417,NaN,NaN,NaN,NaN,42.9808
69394,2016-12-31 18:00:00,NaN,0.450417,NaN,NaN,6.6275,3.1625,9.79001,NaN,NaN,38.8333,24.1750,NaN,NaN,NaN,NaN,NaN


In [6]:
df_condes_verano_test.head()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69395,2017-01-03 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,28.834697,NaN
69396,2017-01-03 14:00:00,NaN,NaN,19.5,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,24.715235,NaN
69397,2017-01-03 15:00:00,NaN,0.836667,13.5,18.0,10.2575,1.7600,12.0175,NaN,NaN,14.9167,28.8167,NaN,NaN,0.0,24.537322,73.6176
69398,2017-01-03 16:00:00,NaN,0.768001,19.5,2.0,10.8350,2.5025,13.3375,NaN,NaN,12.2500,28.2917,NaN,NaN,0.0,17.778006,57.6258
69399,2017-01-03 17:00:00,NaN,0.793750,72.0,10.0,11.5775,2.5850,14.1625,NaN,NaN,14.5000,27.4250,NaN,NaN,0.0,9.862870,51.9867


In [7]:
df_condes_verano.dropna(how='any')

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3


Si elimino todas las que tengan al menos un null se eliminan todos los registros

In [8]:
df_condes_verano_train.loc[:,rows_to_use].dropna(subset=["O3"])#(how="all",O3)

NameError: name 'rows_to_use' is not defined

### PRe PRocesarlos

In [3]:
## representacion
rows_to_use = ["CO","PM10","PM25","NO2","NO","NOX","SO2","WD","RH","TEMP","WS","HCNM","UVA","UVB","O3"]
from keras.preprocessing.sequence import pad_sequences

def estadisticos(timesteps):
    a_utilizar = [np.nanmax]#,np.nanmin,np.nanstd,np.nanmean]
    return np.concatenate([ function(timesteps,axis=0) for function in a_utilizar])

def create_sequences(dataframe,lag=1,dia=True):
    #preprocesamiento asumiendo dataset es una secuencia
    timestep = dataframe.loc[:,"registered_on"].values
    
    indices_orden = []
    diax = timestep[0].split("-")[-1].split(" ")[0]
    aux_indices1 = 0
    aux_indices2 = 0 
    for dato in timestep[1:]:
        nuevo_dia = dato.split("-")[-1].split(" ")[0]
        aux_indices2+=1
        if diax != nuevo_dia:
            diax = nuevo_dia
            #if aux_indices2-aux_indices1==24:
            #    aux_indices2-=1
            indices_orden.append([aux_indices1,aux_indices2])
            aux_indices1 = aux_indices2
    indices_orden.append([aux_indices1,aux_indices2+1])
    data = dataframe.loc[:,rows_to_use].values
    data = np.asarray([data[init:finit] for init,finit in indices_orden])
    
    horas = 24
    
    dataX = []
    dataY =[]
    for t_plus_1 in range(lag,len(data)):
        #se crea el Y (target)
        predecir = data[t_plus_1][:,-1] #todas las ultimas columnas --sequence
        dataY.append(predecir)
        columnasX = [] #se crea el X (inputs) columnas para predecir Y
        for i in np.arange(lag,0,-1): #para los valores anteriores al t_plus_1 durante un lag
            columnasX.append( data[t_plus_1-i] )
        dataX.append(columnasX)
    
    #mascara delete..
    mask_delete = np.ones(len(dataY),dtype=bool)
    if not dia: 
        for i,dato in enumerate(dataY):
            if np.any(np.isnan(dato)): #uno en sequencia---algun nulo
                mask_delete[i] = False #eliminar de training
    else:
        for i,dato in enumerate(dataY):
            if np.all(np.isnan(dato)): #todos en sequencia
                mask_delete[i] = False #eliminar de training
        
    if dia: #calcular estadisticos
        dataX = [[ estadisticos(timestep) for timestep in datito] for datito in dataX] 
    else:
        aux = [ np.concatenate(datito) for datito in dataX] #junta los lag..
        #pad
        mask = np.full(len(rows_to_use) , -1.)
        dataX = pad_sequences(aux, maxlen=lag*horas, dtype='float32', padding='pre', value=mask) #rellena 
    dataY = pad_sequences(dataY, maxlen=horas, dtype='object', padding='post', value=np.nan) #rellena 
    return np.array(dataX)[mask_delete],np.array(dataY)[mask_delete]

In [14]:
lag = 2
dia=False
trainX1_cond, trainY1_cond = create_sequences(df_condes_verano_train, lag,dia)
trainX2_cond, trainY2_cond = create_sequences(df_condes_invierno_train, lag,dia)

testX1_cond, testY1_cond = create_sequences(df_condes_verano_test, lag,dia)
testX2_cond, testY2_cond = create_sequences(df_condes_invierno_test, lag,dia)

print("All arrays shapes")
print("Train verano (input): ",trainX1_cond.shape)
print("Train invierno (input): ",trainX2_cond.shape)
print("Train verano (output): ",trainY1_cond.shape)
print("Train invierno (output): ",trainY2_cond.shape)

All arrays shapes
Train verano (input):  (950, 48, 15)
Train invierno (input):  (967, 48, 15)
Train verano (output):  (950, 24)
Train invierno (output):  (967, 24)


In [20]:
trainX1_cond[0,:,3]

array([ -1.,  -1.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan], dtype=float32)

In [6]:
#o predecir el maximo en etiqueta...
trainY1_cond = np.nanmax(trainY1_cond,axis=1)
trainY2_cond = np.nanmax(trainY2_cond,axis=1)
testY1_cond =  np.nanmax(testY1_cond,axis=1)
testY2_cond =  np.nanmax(testY2_cond,axis=1)
print("Maximo calculado")

Maximo calculado


In [10]:
def trid_to_2d(array,backward=False):
    if backward:
        return array.reshape(array.shape[0]/lag, lag ,array.shape[-1])
    else:
        return array.reshape((np.prod(array.shape[:2]), array.shape[-1]))

In [11]:
trid_to_2d(trid_to_2d(trainX),backward=True).shape

NameError: name 'trainX' is not defined

### Escalar datos y transformar para entrada

In [7]:
#calculo de estadisticos ("fit") MinMaxScaler
class MinMax(object):
    def __init__(self,aux=True):
        self.minimos = None
        self.maximos = None
        
    def fit(self,data):
        self.minimos = np.nanmin(data, axis=(0,1))#, keepdims=True)
        self.maximos = np.nanmax(data ,axis=(0,1))#, keepdims=True)
    
    def transform(self,data):
        #normalize between 0 and 1 every statisctic
        return (data-self.minimos) / (self.maximos-self.minimos)
    
    def inverse_transform(self,data):
        return data*(self.maximos-self.minimos) + self.minimos
    
scaler_model1 = MinMax()
scaler_model1.fit(trainX1_cond)
X_train1_cond = scaler_model1.transform(trainX1_cond)
X_test1_cond = scaler_model1.transform(testX1_cond)

scaler_model2 = MinMax()
scaler_model2.fit(trainX2_cond)
X_train2_cond = scaler_model2.transform(trainX2_cond)
X_test2_cond = scaler_model2.transform(testX2_cond)

X_train1_cond.shape

(2931, 3, 15)

In [8]:
print("Minimos en train: ", np.nanmin(X_train1_cond, axis=(0,1)))
print("Minimos en test: ", np.nanmin(X_test1_cond, axis=(0,1)))

Minimos en train:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Minimos en test:  [6.89655125e-08 2.74869110e-02 2.99625468e-02 1.12001868e-01
 4.98162247e-03 2.49952114e-02            nan            nan
 2.50000000e-01 8.71331507e-02            nan            nan
 0.00000000e+00 5.81265886e-04 8.90017964e-02]


/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: All-NaN slice encountered
  from ipykernel import kernelapp as app


In [9]:
print("Maximos en train: ", np.nanmax(X_train2_cond, axis=(0,1)))
print("Maximos en test: ", np.nanmax(X_test2_cond, axis=(0,1)))

Maximos en train:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Maximos en test:  [0.25981294 0.34102142 0.58563536 0.93545451 0.74557729 0.70256368
        nan        nan 0.91729421 0.85851963        nan        nan
 0.04044388 0.24578276 0.37012987]


/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: All-NaN slice encountered
  from ipykernel import kernelapp as app


In [8]:
mask_value = -1.
def nan_to_num(data,mask_value):
    return np.asarray([[[mask_value if np.isnan(x) else x for x in timestep] for timestep in array] for array in data])

X_train1_cond = nan_to_num(X_train1_cond,mask_value)
X_test1_cond = nan_to_num(X_test1_cond,mask_value)
X_train2_cond = nan_to_num(X_train2_cond,mask_value)
X_test2_cond = nan_to_num(X_test2_cond,mask_value)
print("Mascara agregada")

Mascara agregada


#para el Y tambien...
from sklearn.preprocessing import MinMaxScaler
#o standar
from sklearn.preprocessing import StandardScaler

#scaler_model = MinMaxScaler()
scaler_model = StandardScaler()
scaler_model.fit(trainY1)
y_train1 = scaler_model.transform(trainY1)
y_test1 = scaler_model.transform(testY1)
y_train1.shape

Características del modelo:
* Predecir el máximo de ozono $O_3$ del día siguiente (del período de 24 horas).
* Grano de predicción: Horas o día?
* Cuanta información anterior utilizar? ..3 dias
* Nulos? en $O_3$ o contaminantes?
    * Borrar
    * Mascara

## Modelo

In [54]:
batch_size = 32
epochs = 20
opt = 'adam'
loss_utilizada = 'mse' #?
nhidden = 64
from attention import *

In [64]:
#https://github.com/farizrahman4u/seq2seq
import seq2seq
from seq2seq.models import Seq2Seq
from seq2seq.models import AttentionSeq2Seq


model = Seq2Seq(input_dim=features, input_length=timesteps,hidden_dim=10, output_length=timesteps, output_dim=8)
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_330 (InputLayer)          (None, 3, 15)        0                                            
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 10)        160         input_330[0][0]                  
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__o

In [55]:
#from seq2seq.models import Seq2Seq,AttentionSeq2Seq

timesteps = X_train1_cond.shape[1]
features = X_train1_cond.shape[2] #todo excepto o3

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def attention_multiply(vects):
    encoder, attention = vects
    return K.batch_dot(attention,encoder, axes=1)

def modelo(tipo,dia=True):
    regularization = 0.005
    model = Sequential()
    model.add(Masking(mask_value=mask_value, input_shape=(timesteps, features)))

    if tipo=="sin atencion":
        model.add(GRU(nhidden,return_sequences=True,kernel_regularizer=keras.regularizers.l2(regularization)))
        model.add(GRU(nhidden,return_sequences=True,kernel_regularizer=keras.regularizers.l2(regularization)))
    elif tipo=="atencion":
        model.add(Bidirectional(GRU(nhidden, return_sequences=True,kernel_regularizer=keras.regularizers.l2(regularization))))
        model.add(AttentionDecoder(nhidden, nhidden,kernel_regularizer=keras.regularizers.l2(regularization))) #que son esos  (units, output_dim)
    elif tipo=="sin atencion libreria":
        aux_model = Seq2Seq(input_dim=features, hidden_dim=nhidden, output_length=timesteps, output_dim=nhidden,depth=2)
        #teacherforse=True
        #aux_model = Model(inputs=model.inputs,outputs=aux_model(model.outputs))
        
        layer_aux =  GRU(nhidden, return_sequences=False)(aux_model.outputs)
        layer_aux = Dense(1,activation='linear')(layer_aux) #1 prediccion?
        
        model = Model(inputs=aux_model.inputs,outputs=layer_aux)
        model.compile(loss=loss_utilizada,optimizer=opt)
        model.summary()
        return model
    
    elif tipo=="atencion libreria":
        aux_model = AttentionSeq2Seq(input_dim=features,input_length=timesteps,hidden_dim=nhidden
                                     ,output_length=timesteps,output_dim=nhidden,depth=2)
        model = Sequential()
        model.add(aux_model)
    
    if dia:
        model.add(GRU(nhidden, return_sequences=False,kernel_regularizer=keras.regularizers.l2(regularization)))
        model.add(Dropout(0.3))
        model.add(Dense(1,activation='linear',kernel_regularizer=keras.regularizers.l2(regularization))) #1 prediccion?
    else:
        model.add(TimeDistributed(Dense(1,activation='linear',kernel_regularizer=keras.regularizers.l2(regularization)))) #varias prediccions--a nivel de horario..
        model.add(GlobalMaxPooling1D())
    
    model.compile(loss=loss_utilizada,optimizer=opt)
    model.summary()
    return model

modelo1 = modelo("sin atencion",dia)
modelo2 = modelo("atencion",dia)
#modelo3 = modelo("sin atencion libreria",dia) #no soportan masking
#modelo4 = modelo("atencion libreria",dia)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_23 (Masking)         (None, 3, 15)             0         
_________________________________________________________________
gru_52 (GRU)                 (None, 3, 64)             15360     
_________________________________________________________________
gru_53 (GRU)                 (None, 3, 64)             24768     
_________________________________________________________________
gru_54 (GRU)                 (None, 64)                24768     
_________________________________________________________________
dropout_31 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
Total params: 64,961
Trainable params: 64,961
Non-trainable params: 0
_________________________________________________________________
______

In [110]:
modelo1.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 2931 samples, validate on 195 samples
Epoch 1/20
2931/2931 [==============================] - 16s 5ms/step - loss: 4795.9105 - val_loss: 2463.4797
Epoch 2/20
2931/2931 [==============================] - 3s 1ms/step - loss: 3188.1914 - val_loss: 1612.2063
Epoch 3/20
2931/2931 [==============================] - 3s 1ms/step - loss: 2267.3023 - val_loss: 1067.5342
Epoch 4/20
2931/2931 [==============================] - 3s 1ms/step - loss: 1634.1946 - val_loss: 743.1862
Epoch 5/20
2931/2931 [==============================] - 3s 1ms/step - loss: 1212.6078 - val_loss: 571.3133
Epoch 6/20
2931/2931 [==============================] - 3s 1ms/step - loss: 950.7781 - val_loss: 499.9965
Epoch 7/20
2931/2931 [==============================] - 3s 1ms/step - loss: 796.5459 - val_loss: 486.4146
Epoch 8/20
2931/2931 [==============================] - 3s 1ms/step - loss: 712.0374 - val_loss: 501.9297
Epoch 9/20
2931/2931 [==============================] - 3s 1ms/step - loss: 660.4086 - val_loss:

In [56]:
modelo2.fit(X_train1_cond, trainY1_cond, epochs=epochs, batch_size=batch_size, validation_data=(X_test1_cond, testY1_cond))

Train on 2931 samples, validate on 195 samples
Epoch 1/20
2931/2931 [==============================] - 17s 6ms/step - loss: 6161.6290 - val_loss: 3098.3341
Epoch 2/20
2931/2931 [==============================] - 4s 1ms/step - loss: 4096.7679 - val_loss: 2385.5135
Epoch 3/20
2931/2931 [==============================] - 4s 1ms/step - loss: 3355.7471 - val_loss: 1889.5621
Epoch 4/20
2931/2931 [==============================] - 4s 1ms/step - loss: 2787.2295 - val_loss: 1508.9222
Epoch 5/20
2931/2931 [==============================] - 4s 1ms/step - loss: 2334.8390 - val_loss: 1212.4170
Epoch 6/20
2931/2931 [==============================] - 4s 1ms/step - loss: 1951.3426 - val_loss: 983.7923
Epoch 7/20
2931/2931 [==============================] - 4s 1ms/step - loss: 1645.6302 - val_loss: 811.7584
Epoch 8/20
2931/2931 [==============================] - 4s 1ms/step - loss: 1402.5986 - val_loss: 685.6913
Epoch 9/20
2931/2931 [==============================] - 4s 1ms/step - loss: 1201.8439 - val

In [10]:
## evaluación?
rmse?

from sklearn.metrics import mean_squared_error
def rmse_evaluation(model,input_data,output_data):
    predict = model.predict(input_data)
    #recover data
    predict = scaler_model.inverse_transform(predict)
    output_data = scaler_model.inverse_transform(output_data)
    print("RMSE del modelo en test: ",np.sqrt(mean_squared_error(output_data,predict)))
    return 

def evaluar_modelo(modelito,data):
    mse_test = modelito.evaluate(data[0],data[1])
    print("RMSE del modelo en test: ",np.sqrt(mse_test))
    return np.sqrt(mse_test)
#if es hora.. max
# si es por dia ya predice max

Object `rmse` not found.


In [57]:
evaluar_modelo(modelo2,[X_test1_cond,testY1_cond])       #evaluar en verano unicamente..

195/195 [==============================] - 0s 560us/step
RMSE del modelo en test:  23.028886840777837


23.028886840777837

In [95]:
evaluar_modelo(modelo4,[X_test1,testY1])      

195/195 [==============================] - 0s 630us/step
RMSE del modelo en test:  26.32290688251


26.32290688251

### Mejor Entrenar comunas por separado pero verano e invierno juntos

In [14]:
#stack invierno y verano
X_train_cond = np.concatenate((X_train1_cond,X_train2_cond))
y_train_cond = np.concatenate((trainY1_cond,trainY2_cond))
X_train_cond.shape

(5332, 3, 15)

In [58]:
modelo2_both_cond = modelo("atencion",dia)
modelo2_both_cond.fit(X_train_cond,y_train_cond,epochs=epochs,batch_size=batch_size,validation_data=(X_test1_cond,testY1_cond))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_25 (Masking)         (None, 3, 15)             0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 3, 128)            30720     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 3, 64)             86400     
_________________________________________________________________
gru_58 (GRU)                 (None, 64)                24768     
_________________________________________________________________
dropout_33 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 65        
Total params: 141,953
Trainable params: 141,953
Non-trainable params: 0
_________________________________________________________________
Trai

In [59]:
evaluar_modelo(modelo2_both_cond,[X_test1_cond,testY1_cond])       #evaluar en verano unicamente..

195/195 [==============================] - 0s 561us/step
RMSE del modelo en test:  21.402563555754845


21.402563555754845

### Independencia

In [18]:
df_independencia_verano_train = df_independencia_verano[(df_independencia_verano["registered_on"]<"2017")]
df_independencia_invierno_train = df_independencia_invierno[(df_independencia_invierno["registered_on"]<"2017")]
df_independencia_verano_test = df_independencia_verano[df_independencia_verano["registered_on"]>="2017"]
df_independencia_invierno_test = df_independencia_invierno[df_independencia_invierno["registered_on"]>="2017"]

df_independencia_invierno.tail()

,registered_on,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
60809,2017-08-31 19:00:00,0.75,85.0,33.0,33.22,24.6867,57.9056,NaN,149.499,62.5029,16.1848,1.266580,NaN,0.0,0.0,3.0
60810,2017-08-31 20:00:00,1.10,111.0,36.0,38.79,46.8800,85.6667,NaN,128.815,66.0876,15.2838,0.807589,NaN,0.0,0.0,1.0
60811,2017-08-31 21:00:00,1.32,116.0,34.0,35.44,55.4550,90.8917,NaN,335.223,71.3362,14.0254,0.161541,NaN,0.0,0.0,1.0
60812,2017-08-31 22:00:00,1.75,153.0,67.0,34.11,98.7384,132.8500,NaN,126.665,75.0015,13.2669,0.310322,NaN,0.0,0.0,1.0
60813,2017-08-31 23:00:00,1.73,128.0,43.0,27.97,107.9670,135.9380,NaN,336.886,79.0000,12.4667,0.878486,NaN,0.0,0.0,1.0


In [19]:
lag = 3
dia=True
trainX1_inde, trainY1_inde = create_sequences(df_independencia_verano_train, lag,dia)
trainX2_inde, trainY2_inde = create_sequences(df_independencia_invierno_train, lag,dia)

testX1_inde, testY1_inde = create_sequences(df_independencia_verano_test, lag,dia)
testX2_inde, testY2_inde = create_sequences(df_independencia_invierno_test, lag,dia)

print("All arrays shapes")
print("Train verano (input): ",trainX1_inde.shape)
print("Train invierno (input): ",trainX2_inde.shape)
print("Train verano (output): ",trainY1_inde.shape)
print("Train invierno (output): ",trainY2_inde.shape)

/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: All-NaN slice encountered


All arrays shapes
Train verano (input):  (2924, 3, 15)
Train invierno (input):  (2321, 3, 15)
Train verano (output):  (2924, 24)
Train invierno (output):  (2321, 24)


In [20]:
#o predecir el maximo en etiqueta...
trainY1_inde = np.nanmax(trainY1_inde,axis=1)
trainY2_inde = np.nanmax(trainY2_inde,axis=1)
testY1_inde =  np.nanmax(testY1_inde,axis=1)
testY2_inde =  np.nanmax(testY2_inde,axis=1)
print("Maximo calculado")

Maximo calculado


In [21]:
scaler_model1 = MinMax()
scaler_model1.fit(trainX1_inde)
X_train1_inde = scaler_model1.transform(trainX1_inde)
X_test1_inde = scaler_model1.transform(testX1_inde)

scaler_model2 = MinMax()
scaler_model2.fit(trainX2_inde)
X_train2_inde = scaler_model2.transform(trainX2_inde)
X_test2_inde = scaler_model2.transform(testX2_inde)

X_train1_inde.shape

/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:8: RuntimeWarning: All-NaN slice encountered
/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: All-NaN slice encountered


(2924, 3, 15)

In [22]:
mask_value = -1.
def nan_to_num(data,mask_value):
    return np.asarray([[[mask_value if np.isnan(x) else x for x in timestep] for timestep in array] for array in data])

X_train1_inde = nan_to_num(X_train1_inde,mask_value)
X_test1_inde = nan_to_num(X_test1_inde,mask_value)
X_train2_inde = nan_to_num(X_train2_inde,mask_value)
X_test2_inde = nan_to_num(X_test2_inde,mask_value)
print("Mascara agregada")

Mascara agregada


In [23]:
#stack invierno y verano
X_train_inde = np.concatenate((X_train1_inde,X_train2_inde))
y_train_inde = np.concatenate((trainY1_inde,trainY2_inde))
X_train_inde.shape

(5245, 3, 15)

In [24]:
modelo2_both_inde = modelo("atencion",dia)
modelo2_both_inde.fit(X_train_inde, y_train_inde, epochs=25, batch_size=batch_size, validation_data=(X_test1_inde, testY1_inde))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_5 (Masking)          (None, 3, 15)             0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 3, 128)            30720     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 3, 64)             86400     
_________________________________________________________________
gru_11 (GRU)                 (None, 64)                24768     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 141,953
Trainable params: 141,953
Non-trainable params: 0
_________________________________________________________________
Trai

In [25]:
evaluar_modelo(modelo2_both_inde,[X_test1_inde,testY1_inde])   ##solo verano  

210/210 [==============================] - 0s 466us/step
RMSE del modelo en test:  11.425576062718035


11.425576062718035

In [26]:
print("El promedio entre los dos es: ",np.mean([20.417916143282156,11.425576062718035]))

El promedio entre los dos es:  15.921746103000096
